In [1]:
import tensorflow as tf
import utils
import pandas as pd
import numpy as np
__author__ = "Olivares Castillo José Luis"

tf.__version__

'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
na_dummy = na.drop(na.columns[0],axis=1)
na_vectores1 = np.array(na_dummy)

In [4]:
eval_set = pd.read_csv("../lexiconevaluacion/evaluationset",delimiter=" ",names=["esp","nah"])

In [5]:
eval_set.head()

,esp,nah
0,traer,itqui
1,traer,huica
2,traer,huiqui
3,traer,cui
4,adornar,apana


In [6]:
eval_es = list(set(eval_set["esp"]))

In [7]:
#eval_es

In [8]:
eval_es_index = [int(es[es[0] == palabra].index[0])
                  for palabra in eval_es]

In [9]:
es[es[0] == "pluma"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
2902,pluma,-0.386967000000000005,0.005026,-0.191887000000000002,-0.081058000000000005,-0.211675000000000002,-0.217542999999999986,-0.019026000000000001,0.454046000000000005,0.006309,...,-0.013390000000000001,0.023695999999999998,0.394602000000000008,-0.003221,-0.708667000000000047,0.021819000000000002,0.181774999999999992,0.436242000000000019,-0.162193000000000004,-0.076617000000000005


In [10]:
eval_es_index[0]

1692

In [11]:
eval_es_vectores = utils.get_vectors(es,eval_es_index)

In [12]:
eval_es_vectores.shape

(150, 128)

In [13]:
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in eval_es_index])

In [14]:
sess = tf.Session()

In [15]:
saver = tf.train.import_meta_graph('./models/model2352_gpu/model2352.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/model2352_gpu/'))

INFO:tensorflow:Restoring parameters from ./models/model2352_gpu/model2352.ckpt


In [16]:
graph = tf.get_default_graph()

In [17]:
X = graph.get_tensor_by_name("input/input_es:0")
#y = graph.get_tensor_by_name("input/target_na:0")

In [18]:
#output_NN = graph.get_tensor_by_name("output/xw_plus_b:0")#model1937
output_NN = graph.get_tensor_by_name("xw_plus_b_1:0")
#output_NN = graph.get_tensor_by_name("output_1:0")

In [19]:
feed_dict = {X:test_vectors}


In [20]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (150, 128)


In [21]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]

In [22]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]

In [23]:
es[es[0]=="adquisición"].index

Int64Index([2222], dtype='int64')

In [24]:
na[na[0]=="v"].index

Int64Index([1376], dtype='int64')

In [25]:
eval_es[6]

'teatro'

In [26]:
closest[6]

['tlamahuiz',
 'mahuizo',
 'mahui',
 'imottilih',
 'onimitz',
 'yah',
 'ihcahuaca',
 'cala',
 'itt',
 'tlacacalata']

In [27]:
resultados = {palabra_es:top_10_nah for (palabra_es,top_10_nah) in zip(eval_es,closest)}

In [28]:
esp = list(eval_set["esp"].values)

In [29]:
nah = list(eval_set["nah"].values)

In [30]:
pares_eval = list(zip(esp,nah))

In [31]:
from collections import defaultdict

In [32]:
gold = defaultdict(list)
for palabra_es,palabra_na in pares_eval:
    gold[palabra_es].append(palabra_na)


In [33]:
gold = dict(gold)

In [34]:
gold["amado"]

['tlazo', 'tlazohtla']

In [35]:
resultados["amado"]

['tlazohtla',
 'ama',
 'zquia',
 'ticcua',
 'tlazo',
 'niz',
 'diego',
 'comal',
 'tazohta',
 'federal']

In [36]:
p1 = 0
p5 = 0
p10 = 0
list_esp_eval = (list(resultados.keys()))
hits=list()

not_found = list()


for palabra_gold in list_esp_eval:
    for i in gold[palabra_gold]:
        if i in resultados[palabra_gold]:
            hits.append(resultados[palabra_gold].index(i))
    if hits.__len__() > 0:
        if min(hits) == 0:
            p1 += 1
            p5 += 1
            p10 += 1
        if min(hits) >= 1 and min(hits) <= 5:
            p5 += 1
            p10 += 1
        if min(hits) > 5 and min(hits) <= 10:
            p10 += 1
        print(palabra_gold,min(hits),hits,p1,p5,p10)
    else:
        not_found.append(palabra_gold)
        print(palabra_gold+": NOT FOUND")
   
    hits.clear()

length=list_esp_eval.__len__()
print("\nP@1:",p1/length,"\tP@5:",p5/length,"\tP@10:",p10/length)



brazo: NOT FOUND
tepaneca 1 [1] 0 1 1
cultura 0 [1, 0] 1 2 2
escupir: NOT FOUND
diverso 0 [0] 2 3 3
prestar 0 [0] 3 4 4
teatro 0 [0, 2] 4 5 5
verde 0 [0, 1] 5 6 6
tanto 3 [3] 5 7 7
grande 0 [1, 0] 6 8 8
molcajete 0 [0, 1] 7 9 9
puramente 4 [4] 7 10 10
derecho 1 [1, 2] 7 11 11
echar 1 [1] 7 12 12
caja 0 [0] 8 13 13
muñeca 0 [0] 9 14 14
reunir 1 [1] 9 15 15
barca 0 [0] 10 16 16
tapar 0 [0] 11 17 17
efecto: NOT FOUND
pantalón 0 [0] 12 18 18
vasija 0 [0, 6, 1] 13 19 19
itacates 0 [0] 14 20 20
jadear 0 [0] 15 21 21
yo 0 [0, 8] 16 22 22
dar 0 [1, 0] 17 23 23
leche 3 [3] 17 24 24
lodo 2 [2] 17 25 25
agregar 0 [0, 4] 18 26 26
pasar 0 [0] 19 27 27
tortuga 3 [3] 19 28 28
desear 0 [0, 1, 4] 20 29 29
coyote 0 [0] 21 30 30
establecer: NOT FOUND
aire 0 [0, 5, 1, 1] 22 31 31
salvaje 0 [0] 23 32 32
muerto 0 [1, 0] 24 33 33
paso 0 [0] 25 34 34
adquisición: NOT FOUND
traer 1 [1, 8, 3] 25 35 35
descanso 0 [0] 26 36 36
adornar 2 [3, 2] 26 37 37
amado 0 [4, 0] 27 38 38
mesero 0 [0] 28 39 39
presencia: NOT 

In [37]:
not_found.__len__()

18

In [38]:
len(list(resultados.keys()))

150

In [39]:
resultados_gold = dict()
for k,v in resultados.items():
    resultados_gold[k] = {"gold":gold[k],"resultados":v}

In [40]:
na[na[0]=="español"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
1342,español,-0.059672000000000003,-0.354704000000000019,-0.177631000000000011,0.082270999999999983,0.294655,0.32784000000000002,-0.148658000000000012,-0.437325999999999993,-0.064614000000000019,...,0.145312999999999998,0.263815000000000022,-0.113244000000000011,0.194174999999999986,0.199001000000000011,0.957316999999999974,-0.743376999999999843,0.100334000000000007,0.050588000000000001,-0.401527000000000023


In [41]:
for res in not_found:
    print(res+":",resultados_gold[res],end="\n"*2)

brazo: {'gold': ['aca'], 'resultados': ['cuecue', 'icemel', 'tonehua', 'notelpo', 'ticte', 'tepotz', 'cocone', 'pahpaqui', 'milacatzo', 'tz']}

escupir: {'gold': ['chicha', 'chichi'], 'resultados': ['yb', 'zcuco', 'im', 'dral', 'diego', 'huiz', 'rosario', 'tepahpaquiltia', 'secretario', 'zque']}

efecto: {'gold': ['nel'], 'resultados': ['tlaocox', 'nehnemi', 'oquich', 'ihtotia', 'mixco', 'tz', 'timamalihuac', 'iuhquin', 'oceloquichtle', 'ixeo']}

establecer: {'gold': ['tlalia', 'chan'], 'resultados': ['ez', 'lique', 'itoa', 'cauh', 'iuh', 'acique', 'nahuati', 'tepe', 'e', 'ayo']}

adquisición: {'gold': ['quipiaz', 'tocococauh', 'cococauh', 'cococa'], 'resultados': ['oticma', 'chiquiuh', 'mel', 'amech', 'lahtol', 'ceuh', 'amaneapan', 'immoma', 'pilca', 'poyox']}

presencia: {'gold': ['ix'], 'resultados': ['mopechteccac', 'tlacuauh', 'tzinco', 'huexo', 'quimolhui', 'melauh', 'tona', 'uad', 'tlazoh', 'eltilitiuh']}

valer: {'gold': ['patio'], 'resultados': ['quexquich', '1', 'ton', 'mama'

In [42]:
resultados_gold["sabor"]

{'gold': ['iahhuui', 'mati'],
 'resultados': ['mati',
  'lan',
  'tro',
  'iahhuui',
  'xinach',
  'ixmachilia',
  'ixmati',
  'chil',
  'ahnozo',
  'machi']}

In [43]:
for k,v in resultados_gold.items():
    print(k,":",v,end="\n"*2)

brazo : {'gold': ['aca'], 'resultados': ['cuecue', 'icemel', 'tonehua', 'notelpo', 'ticte', 'tepotz', 'cocone', 'pahpaqui', 'milacatzo', 'tz']}

tepaneca : {'gold': ['eca'], 'resultados': ['colhuaca', 'eca', 'popolo', 'motlanahuatili', 'cocox', 'culhuacan', 'rtes', 'oquim', 'acique', 'mexica']}

cultura : {'gold': ['tlamatiliz', 'cultura'], 'resultados': ['cultura', 'tlamatiliz', 'toyalhuayo', 'nahuatlahtol', 'iayah', 'moyacanilia', 'lahtol', 'huehue', 'moteihtitili', 'potz']}

escupir : {'gold': ['chicha', 'chichi'], 'resultados': ['yb', 'zcuco', 'im', 'dral', 'diego', 'huiz', 'rosario', 'tepahpaquiltia', 'secretario', 'zque']}

diverso : {'gold': ['nepapan'], 'resultados': ['nepapan', 'mota', 'motemilih', 'tatemp', 'tenanco', 'tata', 'tlalli', 'huipan', 'izqui', 'caxtillan']}

prestar : {'gold': ['ehuia'], 'resultados': ['ehuia', 'onite', 'icni', '19', 'coco', 'jo', 'gr', 'xinechmotlapohpolhuili', 'fe', 'george']}

teatro : {'gold': ['tlamahuiz', 'mahui'], 'resultados': ['tlamahuiz',